In [31]:
%pip install pandas
%pip install openpyxl

import pandas as pd
from pathlib import Path
import requests
import os


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Downloading XLS files from council website

In [38]:


# Create list of target URLs to download
classified_urls_df = pd.read_csv('../wayback/url_classified.csv')
classified_urls_df['target_url'] = classified_urls_df['wayback_download_url'].str.split('id_/').str[1]

list_new_businesses = classified_urls_df[classified_urls_df['category'] == 'new_businesses']['target_url'].dropna().unique()
list_accounts_no_relief = classified_urls_df[classified_urls_df['category'] == 'accounts_no_relief']['target_url'].dropna().unique()
list_account_relief = classified_urls_df[classified_urls_df['category'] == 'accounts_relief']['target_url'].dropna().unique()
list_accouts_closed = classified_urls_df[classified_urls_df['category'] == 'accounts_closed']['target_url'].dropna().unique()
list_account_current_in_credit = classified_urls_df[classified_urls_df['category'] == 'accounts_current_in_credit']['target_url'].dropna().unique()

# Extra urls

list_new_businesses_extra = ['https://www.stalbans.gov.uk/sites/default/files/attachments/NNDR_New%20Businesses%20December%202024%20to%20May%202025.xlsx']
list_accounts_no_relief_extra = ['https://www.stalbans.gov.uk/sites/default/files/attachments/NNDR%20Accounts%20With%20No%20Relief_June%202025.xlsx']
list_account_relief_extra = ['https://www.stalbans.gov.uk/sites/default/files/attachments/NNDR%20Accounts%20with%20a%20relief_June%202025.xlsx']
list_accouts_closed_extra = ['https://www.stalbans.gov.uk/sites/default/files/attachments/NNDR%20Closed%20Accounts%20in%20Credit_30.05.2025.xlsx']
list_account_current_in_credit_extra = ['https://www.stalbans.gov.uk/sites/default/files/attachments/NNDR%20Current%20Accounts%20in%20Credit_30.05.2025.xlsx']

# Download raw files to local
save_dirs = ['new_businesses', 'accounts_no_relief', 'accounts_relief', 'accounts_closed', 'accounts_current_in_credit']

config = {
    'new_businesses': {
        'urls': list(list_new_businesses) + list_new_businesses_extra,
        'save_dir': 'new_businesses',
    },
    'accounts_no_relief': {
        'urls': list(list_accounts_no_relief) + list_accounts_no_relief_extra,
        'save_dir': 'accounts_no_relief',
    },
    'accounts_relief': {
        'urls': list(list_account_relief) + list_account_relief_extra,
        'save_dir': 'accounts_relief',
    },
    'accounts_closed': {
        'urls': list(list_accouts_closed) + list_accouts_closed_extra,
        'save_dir': 'accounts_closed',
    },
    'accounts_current_in_credit': {
        'urls': list(list_account_current_in_credit) + list_account_current_in_credit_extra,
        'save_dir': 'accounts_current_in_credit',
    },
}

for save_dir in save_dirs:
    os.makedirs('raw_files/' + save_dir, exist_ok=True)

for category, config in config.items():
    for url in config['urls']:
        raw_file_name = url.split('/')[-1]
        print(f'Downloading {raw_file_name} from {url}')
        # Stream the content and write it to disk
        with requests.get(url, stream=True) as r:
            r.raise_for_status()  # raises exception if download failed
            with open('raw_files/' + config['save_dir'] + '/' + raw_file_name, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)


# Sanitization

### New businesses

In [41]:


config = {
    'new_businesses': {
        'folder_path': 'raw_files/new_businesses',
        'column_mapping': {
            "Current Analysis Code Description": "current_analysis_code_description",
            "Primary Liable party name": "primary_liable_party_name",
            "Property Reference Number": "property_reference_number",
            "Full Property Address": "full_property_address",
            "Liable Responsibility Start Date": "liable_responsibility_start_date",
            "Current Rateable Value": "current_rateable_value",
        },
        'desired_order': [
            "current_analysis_code_description",
            "primary_liable_party_name",
            "property_reference_number",
            "full_property_address",
            "liable_responsibility_start_date",
            "current_rateable_value", 
            "source_file"
        ]
    },
    'accounts_closed': {
        'folder_path': 'raw_files/accounts_closed',
        'column_mapping': {
            'Property Reference Number' : 'property_reference_number',
            'Primary Liable party name' : 'primary_liable_party_name',
            'Full Property Address' : 'full_property_address',
            'Account Start date' : 'account_start_date',
            'Account End date' : 'account_end_date',
            'Outstanding Debt' : 'outstanding_debt'
        },
        'desired_order': [
           'property_reference_number',
            'primary_liable_party_name',
            'full_property_address',
            'account_start_date',
            'account_end_date',
            'outstanding_debt',
            'source_file'
        ]
    },
    'accounts_relief': {
        'folder_path': 'raw_files/accounts_relief',
        'column_mapping': {
            'Property Reference Number' : 'property_reference_number',
            'Primary Liable party name' : 'primary_liable_party_name',
            'Full Property Address' : 'full_property_address',
            'Liable Responsibility Start Date' : 'liable_responsibility_start_date',
            'Current Rateable Value' : 'current_rateable_value',
            'Current Relief Type' : 'current_relief_type',
            'Current Relief Type Description' : 'current_relief_type_description',
            'Current Relief Award Start Date' : 'current_relief_award_start_date',
            'Current Relief Award Perc Awarded' : 'current_relief_award_perc_awarded'
        },
        'desired_order': [
            'property_reference_number',
            'primary_liable_party_name',
            'full_property_address',
            'liable_responsibility_start_date',
            'current_rateable_value',
            'current_relief_type',
            'current_relief_type_description',
            'current_relief_award_start_date',
            'current_relief_award_perc_awarded',
            'source_file'
        ]
    },
    "accounts_no_relief": {
        'folder_path': 'raw_files/accounts_no_relief',
        'column_mapping': {
            'Property Reference Number': 'property_reference_number',
            'Primary Liable party name': 'primary_liable_party_name',
            'Full Property Address': 'full_property_address',
            'Liable Responsibility Start Date': 'liable_responsibility_start_date',
            'Current Rateable Value': 'current_rateable_value',
            'Billing Period': 'billing_period',
            'A/C Summary Current Liability': 'a_c_summary_current_liability'
        },
        'desired_order': [
            'property_reference_number',
            'primary_liable_party_name',
            'full_property_address',
            'liable_responsibility_start_date',
            'current_rateable_value',
            'billing_period',
            'a_c_summary_current_liability',
            'source_file'
        ]
    }
}

def concat_dataframes(folder_path, column_mapping, desired_order):
    print(f"Concatenating dataframes from {folder_path}")
    dfs = []
    for file_path in Path(folder_path).glob('*.xlsx'):
        print(f'Processing {file_path}')
        # Load Excel file into pandas
        df = pd.read_excel(file_path, sheet_name=0)  # sheet_name=0 loads first sheet
        print(f"Number of rows: {len(df)}")
        # Rename columns using the mapping
        df = df.rename(columns=column_mapping)
        # Add filename column for reference
        df['source_file'] = file_path.name

        existing_columns = [col for col in desired_order if col in df.columns]
        df = df[existing_columns]
        # Store the dataframe
        dfs.append(df)

    result_df = pd.concat(dfs, ignore_index=True)
    print(f"Resulting dataframe has {len(result_df)} rows")

    return result_df



new_businesses_consolidated_df = concat_dataframes(
    'raw_files/new_businesses',
    config['new_businesses']['column_mapping'],
    config['new_businesses']['desired_order']
)

businesses_closed_consolidated_df = concat_dataframes(
    'raw_files/accounts_closed',
    config['accounts_closed']['column_mapping'],
    config['accounts_closed']['desired_order']
)

accounts_relief_consolidated_df = concat_dataframes(
    'raw_files/accounts_relief',
    config['accounts_relief']['column_mapping'],
    config['accounts_relief']['desired_order']
)

accounts_no_relief_consolidated_df = concat_dataframes(
    'raw_files/accounts_no_relief',
    config['accounts_no_relief']['column_mapping'],
    config['accounts_no_relief']['desired_order']
)










Concatenating dataframes from raw_files/new_businesses
Processing raw_files/new_businesses/NNDR%20New%20Businesses%20from%20June%20to%20October%202023.xlsx
Number of rows: 99
Processing raw_files/new_businesses/NNDR_New%20Businesses%20from%20October20%20to%20January21.xlsx
Number of rows: 74
Processing raw_files/new_businesses/New%20Businesses%20July-Sept20_0.xlsx
Number of rows: 71
Processing raw_files/new_businesses/New%20Businesses%20from%20October%202021%20to%20March%202022.xlsx
Number of rows: 130
Processing raw_files/new_businesses/New%20Businesses%20from%20April22%20to%20August22.xlsx
Number of rows: 113
Processing raw_files/new_businesses/NNDR_New%20Businesses%20from%20Nov23%20to%20March24.xlsx
Number of rows: 91
Processing raw_files/new_businesses/New%20Businesses%20from%20July%20to%20September21.xlsx
Number of rows: 70
Processing raw_files/new_businesses/NNDR_New%20Businesses%20December%202024%20to%20May%202025.xlsx
Number of rows: 118
Processing raw_files/new_businesses/NNDR

In [42]:
# Save consolidated dataframes to CSV files
import os

# Create consolidated_files directory if it doesn't exist
os.makedirs('consolidated_files', exist_ok=True)

# Save each consolidated dataframe to CSV
print("Saving consolidated dataframes to CSV files...")

new_businesses_consolidated_df.to_csv('consolidated_files/new_businesses_consolidated.csv', index=False)
print(f"Saved new_businesses_consolidated.csv with {len(new_businesses_consolidated_df)} rows")

businesses_closed_consolidated_df.to_csv('consolidated_files/accounts_closed_consolidated.csv', index=False)
print(f"Saved accounts_closed_consolidated.csv with {len(businesses_closed_consolidated_df)} rows")

accounts_relief_consolidated_df.to_csv('consolidated_files/accounts_relief_consolidated.csv', index=False)
print(f"Saved accounts_relief_consolidated.csv with {len(accounts_relief_consolidated_df)} rows")

accounts_no_relief_consolidated_df.to_csv('consolidated_files/accounts_no_relief_consolidated.csv', index=False)
print(f"Saved accounts_no_relief_consolidated.csv with {len(accounts_no_relief_consolidated_df)} rows")

print("✅ All consolidated CSV files saved successfully!")


Saving consolidated dataframes to CSV files...
Saved new_businesses_consolidated.csv with 1307 rows
Saved accounts_closed_consolidated.csv with 2683 rows
Saved accounts_relief_consolidated.csv with 28209 rows
Saved accounts_no_relief_consolidated.csv with 27009 rows
✅ All consolidated CSV files saved successfully!
